### 1️⃣ Importaciones y configuración

In [ ]:
import sys
import os

BASE_DIR = os.path.abspath(os.path.join(os.path.dirname("__file__"), "..", ".."))
OUTPUT_DIR = os.path.join(BASE_DIR, "output")

TRAIN_CSV = os.path.join(OUTPUT_DIR, "train_processed.csv")
VAL_CSV   = os.path.join(OUTPUT_DIR, "val_processed.csv")
TEST_CSV  = os.path.join(OUTPUT_DIR, "test_processed.csv")

sys.path.insert(0, BASE_DIR)

from src.part_2.main import run_training
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd


### 2️⃣ Funciones de evaluación de métricas

In [ ]:
def mape(y_true, y_pred):
    """Mean Absolute Percentage Error"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape_val = mape(y_true, y_pred)
    return {
        "MSE": float(mse),
        "MAE": float(mae),
        "MAPE": float(mape_val)
    }


### 4️⃣ Cargar datos usando las variables del notebook


In [ ]:
import pandas as pd

hyperparameter_sets = [
    {"activation": "relu",    "layers": [277, 32, 16, 1],    "epochs": 150, "lr": 0.001,  "momentum": 0.9},
    {"activation": "relu",    "layers": [277, 64, 32, 1],    "epochs": 200, "lr": 0.0005, "momentum": 0.9},
    {"activation": "tanh",    "layers": [277, 20, 10, 1],    "epochs": 200, "lr": 0.05,   "momentum": 0.9},
    {"activation": "tanh",    "layers": [277, 40, 20, 1],    "epochs": 250, "lr": 0.02,   "momentum": 0.8},
    {"activation": "sigmoid", "layers": [277, 20, 10, 1],    "epochs": 70,  "lr": 0.001,  "momentum": 0.9},
    {"activation": "sigmoid", "layers": [277, 50, 25, 1],    "epochs": 120, "lr": 0.005,  "momentum": 0.7},
    {"activation": "linear",  "layers": [277, 1],            "epochs": 200, "lr": 1e-6,   "momentum": 0},
    {"activation": "relu",    "layers": [277, 128, 64, 1],   "epochs": 250, "lr": 0.0003, "momentum": 0.9},
    {"activation": "tanh",    "layers": [277, 64, 32, 1],    "epochs": 150, "lr": 0.01,   "momentum": 0.7},
    {"activation": "sigmoid", "layers": [277, 10, 5, 1],     "epochs": 80,  "lr": 0.002,  "momentum": 0.8},
]

results = []

for i, params in enumerate(hyperparameter_sets, start=1):
    print(f"\n=== Running configuration {i} / {len(hyperparameter_sets)} ===")
    print(params)

    nn, pid_test, y_test, preds, trainer_obj = run_training(
        activation=params["activation"],
        layers=params["layers"],
        epochs=params["epochs"],
        lr=params["lr"],
        momentum=params["momentum"],
        CSV_TRAIN=TRAIN_CSV,
        CSV_VAL=VAL_CSV,
        CSV_TEST=TEST_CSV,
        verbose=False
    )

    metrics = evaluate_model(y_test, preds)

    results.append({
        "layers": params["layers"],
        "epochs": params["epochs"],
        "lr": params["lr"],
        "momentum": params["momentum"],
        "activation": params["activation"],
        "MAPE": metrics["MAPE"],
        "MAE":  metrics["MAE"],
        "MSE":  metrics["MSE"],
    })

df_results = pd.DataFrame(results)
df_results


